In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from pandarallel import pandarallel
from sklearn.preprocessing import OneHotEncoder
import pickle
import time

In [2]:
# pandarallel.initialize(progress_bar=True,nb_workers=15,shm_size_mb=3000)

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
def parallelize_dataframe(df, func, n_cores=10):
    df_split = np.array_split(df, n_cores)
    pool = multiprocessing.Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [5]:
d = pd.read_csv('../semantic_f_w2v_g2v_upd_wfilter.csv')

In [6]:
d.head()

,query_id,query,table_path,table_id,parsedTable,w2v_embd_table,rel,w2v_embd_query,w2v_early_fusion,w2v_late_fusion,w2v_late_fusion_max,w2v_late_fusion_avg,w2v_late_fusion_sum,query_entites,query_entities_lst,table_entities,g2v_embd_table,g2v_embd_query,g2v_early_fusion,g2v_late_fusion,g2v_late_fusion_max,g2v_late_fusion_avg,g2v_late_fusion_sum,query_entities_oh,table_entities_filter
0,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0875-680,"['experian', 'score', 'expected', 'annual', 'l...",[[-0.09472656 -0.51171875 0.05810547 ... 0.3...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.572375,[ 0.07764402 0.04398614 0.062119 0.131361...,1.0,0.132950,18.081264,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Rate_risk', 'Inflation_targeting', 'Overnigh...","['Cash_advance', 'Subprime_lending', 'Credit_c...",[[-0.10551962 0.16365984 -0.10385874 ... 0.0...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.630084,[0.8126399 0.5829352 0.8141425 ... 0.798044...,0.857562,0.585746,10747.270508,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,"['Cash_advance', 'Subprime_lending', 'Credit_c..."
1,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1020-619,"['annual', 'inflation', 'rates', 'aug', 'sep',...",[[ 0.0390625 0.01586914 -0.07373047 ... 0.1...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.681248,[ 0.13136129 0.06132873 0.00324154 0.019790...,1.0,0.160487,10.271143,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Rate_risk', 'Inflation_targeting', 'Overnigh...","['Ultra-Tories', 'Zero_interest-rate_policy', ...",[[-0.5667573 0.05929473 -0.35778925 ... -0.3...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.769993,[0.5664706 0.6671674 0.63320464 ... 0.680116...,0.899880,0.620371,12417.335938,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,"['Ultra-Tories', 'Zero_interest-rate_policy', ..."
2,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-531,"['regret', 'interest', 'rates', 'rise', 'stati...",[[ 1.9287109e-02 7.5195312e-02 3.4179688e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.606677,[ 0.09074079 0.07134737 0.00324154 0.122679...,1.0,0.145743,12.825369,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Rate_risk', 'Inflation_targeting', 'Overnigh...","['Expected_utility_hypothesis', 'Loss_function...",[[-0.01618436 0.36752364 -0.41611332 ... -0.2...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.879135,[0.5724195 0.57001764 0.9345857 ... 0.456453...,1.000000,0.646446,11321.862305,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,"['Expected_utility_hypothesis', 'Loss_function..."
3,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-530,"['return', 'interest', 'rates', 'rise', 'stati...",[[ 5.1269531e-02 1.2597656e-01 -1.3769531e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.617735,[ 0.0134912 0.07134737 0.00324154 0.122679...,1.0,0.140305,13.469313,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Rate_risk', 'Inflation_targeting', 'Overnigh...","['Allan_Saint-Maximin', 'Maximin_Coia', 'Saint...",[[-0.28710952 0.19961576 -0.22583863 ... 0.2...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.866539,[0.77406514 0.61262506 0.56347716 ... 0.734163...,1.000000,0.638918,11722.868164,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,"['Allan_Saint-Maximin', 'Maximin_Coia', 'Saint..."
4,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1000-57,"['security', 'nominal', 'interest', 'rates', '...",[[ 1.8920898e-02 1.8554688e-01 4.3945312e-02...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.586386,[ 9.76208225e-02 4.97327670e-02 7.13473707e-...,1.0,0.139647,18.433437,{'3': {'entity': '<dbpedia:Permanent_interest_...,"['Rate_risk', 'Inflation_targeting', 'Overnigh...","['Quantitative_easing', 'Inflation_derivative'...",[[-0.08840087 0.19392557 -0.24297218 ... -0.3...,[[-0.16757718 0.09818904 -0

In [7]:
shrt_abs = pd.read_csv('./list_short_abstracts.csv')

In [8]:
all_ent = list(shrt_abs.iloc[:,1].unique())

In [9]:
categ = pd.read_csv('./list_category_labels.csv')

In [10]:
all_categ = list(categ.iloc[:,1].unique())

In [11]:
ent_w_cat = pd.read_csv('./list_article_categories_grouped.csv')

In [12]:
# ent_w_cat['2_cat_lst'] = ent_w_cat['2_cat'].parallel_apply(lambda x: x.split('_/:/_'))

def modif_ent_w_cat(ent_w_cat):
    ent_w_cat['2_cat_lst'] = ent_w_cat['2_cat'].apply(lambda x: x.split('_/:/_'))
    return ent_w_cat

ent_w_cat = parallelize_dataframe(ent_w_cat, modif_ent_w_cat, 15)

In [13]:
ent_w_cat.head()

,0_en,2_cat,2_cat_lst
0,!!!,Rock_music_groups_from_California_/:/_Musical_...,"[Rock_music_groups_from_California, Musical_gr..."
1,!!!_(album),!!!_albums_/:/_2001_debut_albums_/:/_Gold_Stan...,"[!!!_albums, 2001_debut_albums, Gold_Standard_..."
2,!!Destroy-Oh-Boy!!,1993_debut_albums_/:/_New_Bomb_Turks_albums_/:...,"[1993_debut_albums, New_Bomb_Turks_albums, Cry..."
3,!!M,Instant_messaging_clients,[Instant_messaging_clients]
4,!Action_Pact!,Musical_groups_established_in_1981_/:/_Musical...,"[Musical_groups_established_in_1981, Musical_g..."


In [14]:
len(all_categ),len(all_ent)

(1321139, 4641890)

In [15]:
d['table_entities_filter'] = d['table_entities_filter'].apply(eval)
d['query_entities_lst'] = d['query_entities_lst'].apply(eval)

In [16]:
def get1hot(entities):
    en = np.array(entities).reshape(-1,1)
    return enc.transform(en).toarray()

In [17]:
enc = OneHotEncoder()
enc.fit(np.array(all_ent).reshape(-1,1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [18]:
# %%time
# l = {}
# l_ = list(d.query_entities_lst.unique())
# for i in range(len(l_)):
#     print(l_[i])
#     l[i+1] = get1hot(l_[i])

In [19]:
with open('./query_ent_oh_dict.pkl', 'rb') as f:
    l = pickle.load(f)

In [20]:
d['query_entities_oh'] = d.apply(lambda x: l[x['query_id']],axis=1)

In [21]:
# S = set(all_ent)
# d['table_entities_filter'] = d.table_entities.parallel_apply(lambda x: list(set(x).intersection(S)))

# def modif_tab_en_filter(d):
#     d['table_entities_filter'] = d.table_entities.apply(lambda x: [i for i in x if i in all_ent])
#     return d

# d = parallelize_dataframe(d, modif_tab_en_filter, 15)

In [22]:
# d['table_entities_oh'] = d.table_entities_filter.parallel_apply(lambda x: get1hot(x))

In [23]:
# %%time
# no_of_p = len(l1)//100
# df_split = np.array_split(l1, no_of_p)
# pool = multiprocessing.Pool(no_of_p)
# df = np.concatenate(pool.map(get1hot, df_split),axis=0)

In [24]:
# with open('./query_ent_oh_dict.pkl', 'wb') as f:
#         pickle.dump(l, f, pickle.HIGHEST_PROTOCOL)

In [25]:
a = list(d.table_entities_filter)
for i in range(len(a)):
    a[i] = np.array(a[i])

l1 = []
for i in a:
    for j in i:
        l1.append(j)
l1 = list(set(l1))

In [26]:
l2 = get1hot(l1)

In [27]:
def get_table_ohe(l):
    return np.array([l2[l1.index(i)] for i in l])

In [31]:
def late_fusion(table,query):
    s = []
    for i in query:
        for j in table:
            sim = cosine_similarity(i.reshape(1,-1),j.reshape(1,-1))
            s.append(sim)
    s = np.array(s).reshape(-1)
    return s

In [48]:
def early_fusion(table,query):
    a = np.average(table,axis=0).reshape(1,-1)
    b = np.average(query,axis=0).reshape(1,-1)
    sim = cosine_similarity(a,b)
    return sim.reshape(-1)[0]

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
def compute(row):
    table_en_oh = np.array([l2[l1.index(i)] for i in row['table_entities_filter']])
    query_en_ih = row['query_entities_oh']
    return early_fusion(table_en_oh,query_en_ih), late_fusion(table_en_oh,query_en_ih)

In [56]:
a = d.iloc[:2].apply(lambda x: compute(x),axis=1)

In [65]:
t = time.time()
# d.iloc[:10].apply(lambda x: compute(x),axis=1)
# d.iloc[:10].table_entities_filter.apply(lambda x: get_table_ihe(x))
def filter_en(d):
    d['table_entities_oh'] = d.apply(lambda x: compute(x),axis=1)
    return d

z1 = parallelize_dataframe(d, filter_en, 50)
print(time.time()-t)


9549.274335861206


In [77]:
z1.head()

,query_id,query,table_path,table_id,parsedTable,w2v_embd_table,rel,w2v_embd_query,w2v_early_fusion,w2v_late_fusion,w2v_late_fusion_max,w2v_late_fusion_avg,w2v_late_fusion_sum,query_entites,query_entities_lst,table_entities,g2v_embd_table,g2v_embd_query,g2v_early_fusion,g2v_late_fusion,g2v_late_fusion_max,g2v_late_fusion_avg,g2v_late_fusion_sum,table_entities_filter,boe_early_fusion,boe_late_fusion
0,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0875-680,"['experian', 'score', 'expected', 'annual', 'l...",[[-0.09472656 -0.51171875 0.05810547 ... 0.3...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.572375,[ 0.07764402 0.04398614 0.062119 0.131361...,1.0,0.132950,18.081264,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Rate_risk, Inflation_targeting, Overnight_pol...","['Cash_advance', 'Subprime_lending', 'Credit_c...",[[-0.10551962 0.16365984 -0.10385874 ... 0.0...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.630084,[0.8126399 0.5829352 0.8141425 ... 0.798044...,0.857562,0.585746,10747.270508,"[Cash_advance, Subprime_lending, Credit_card_b...",0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1020-619,"['annual', 'inflation', 'rates', 'aug', 'sep',...",[[ 0.0390625 0.01586914 -0.07373047 ... 0.1...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.681248,[ 0.13136129 0.06132873 0.00324154 0.019790...,1.0,0.160487,10.271143,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Rate_risk, Inflation_targeting, Overnight_pol...","['Ultra-Tories', 'Zero_interest-rate_policy', ...",[[-0.5667573 0.05929473 -0.35778925 ... -0.3...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.769993,[0.5664706 0.6671674 0.63320464 ... 0.680116...,0.899880,0.620371,12417.335938,"[Ultra-Tories, Zero_interest-rate_policy, Inte...",0.065938,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-531,"['regret', 'interest', 'rates', 'rise', 'stati...",[[ 1.9287109e-02 7.5195312e-02 3.4179688e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.606677,[ 0.09074079 0.07134737 0.00324154 0.122679...,1.0,0.145743,12.825369,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Rate_risk, Inflation_targeting, Overnight_pol...","['Expected_utility_hypothesis', 'Loss_function...",[[-0.01618436 0.36752364 -0.41611332 ... -0.2...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.879135,[0.5724195 0.57001764 0.9345857 ... 0.456453...,1.000000,0.646446,11321.862305,"[Expected_utility_hypothesis, Loss_function, V...",0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-0288-530,"['return', 'interest', 'rates', 'rise', 'stati...",[[ 5.1269531e-02 1.2597656e-01 -1.3769531e-01...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.617735,[ 0.0134912 0.07134737 0.00324154 0.122679...,1.0,0.140305,13.469313,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Rate_risk, Inflation_targeting, Overnight_pol...","['Allan_Saint-Maximin', 'Maximin_Coia', 'Saint...",[[-0.28710952 0.19961576 -0.22583863 ... 0.2...,[[-0.16757718 0.09818904 -0.15539637 ... 0.0...,0.866539,[0.77406514 0.61262506 0.56347716 ... 0.734163...,1.000000,0.638918,11722.868164,"[Allan_Saint-Maximin, Maximin_Coia, Saint-Maxi...",0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,world interest rates Table,/home/vibhav/table2vec/tables_redi2_1/re_table...,table-1000-57,"['security', 'nominal', 'interest', 'rates', '...",[[ 1.8920898e-02 1.8554688e-01 4.3945312e-02...,0,[[-6.3964844e-02 6.8359375e-02 2.2460938e-01...,0.586386,[ 9.76208225e-02 4.97327670e-02 7.13473707e-...,1.0,0.139647,18.433437,{'3': {'entity': '<dbpedia:Permanent_interest_...,"[Rate_risk, Inflation_targeting, Overnight_pol...","['Quantitative_easing', 'Inflation_derivative'...",[[-0.08840087 0.1

In [76]:
z1.drop(columns=['w2v_embd_table','w2v_embd_query'],inplace=True)

In [ ]:
z